In [1]:
    import hopsworks
    import pandas as pd
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report
    import seaborn as sns
    from matplotlib import pyplot
    from hsml.schema import Schema
    from hsml.model_schema import ModelSchema
    import joblib
    import os

In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/212121
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
wine_fg = fs.get_feature_group(name="wine_quality", version=1)
query = wine_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="wine_quality",
                                  version=1,
                                  description="Read from Wine Quality dataset",
                                  labels=["quality"],
                                  query=query)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/212121/fs/212040/fv/wine_quality/version/1


In [4]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/212121/featurestores/212040/featureview/wine_quality/version/1/trainingdatasets). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, body: b'{"errorCode":270186,"usrMsg":"Training datasets quota reached for Project id2223_mldl. Current: 100 Max: 100","devMsg":"Training datasets quota reached for Project id2223_mldl. Current: 100 Max: 100","errorMsg":"Could not create training dataset"}', error code: 270186, error msg: Could not create training dataset, user msg: Training datasets quota reached for Project id2223_mldl. Current: 100 Max: 100

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)
X_test

In [ ]:
X_train

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(n_estimators=1000, max_depth=7, min_weight_fraction_leaf=0.00001)
# model.fit(X_train, y_train)

In [ ]:
# from sklearn.ensemble import AdaBoostClassifier
# model = AdaBoostClassifier(n_estimators=150, learning_rate=0.1)
# model.fit(X_train, y_train)

In [ ]:
# from sklearn.tree import DecisionTreeClassifier 
# model = DecisionTreeClassifier()
# model.fit(X_train, y_train)

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=4000, learning_rate=0.1, depth=4) #3000, 0.01, 5 == 0.56
model.fit(X_train, y_train)

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)
# y_train

In [ ]:
# y_test = le.transform(y_test)
# y_test

In [ ]:
# # fit model no training data
# import xgboost as xgb
# model = xgb.XGBClassifier(n_estimators = 1300, learning_rate = 0.1, max_depth = 3)
# model.fit(X_train, y_train)

In [ ]:
#check X train
y_pred = model.predict(X_train)
results = confusion_matrix(y_train, y_pred)
#consfision matrix
df_cm = pd.DataFrame(results, [3 ,4 ,5 ,6, 7, 8, 9],
                     [3 ,4 ,5, 6, 7, 8, 9])
cm = sns.heatmap(df_cm, annot=True)
fig = cm.get_figure()

In [ ]:
# Evaluate model performance using the features from the test set (X_test)
y_pred = model.predict(X_test)
# Compare predictions (y_pred) with the labels in the test set (y_test)
metrics = classification_report(y_test, y_pred, output_dict=True)
results = confusion_matrix(y_test, y_pred)
results

In [ ]:
accuracy = accuracy_score(y_test, y_pred, normalize=True)
accuracy

In [ ]:
metrics = classification_report(y_test, y_pred, output_dict=True)
results = confusion_matrix(y_test, y_pred)
results

In [ ]:
#print(metrics)
metrics_df = pd.DataFrame(metrics).transpose()
metrics_df

In [ ]:
# Create the confusion matrix as a figure, we will later store it as a PNG image file
df_cm = pd.DataFrame(results, [3 ,4 ,5 ,6, 7, 8, 9],
                     [3 ,4 ,5, 6, 7, 8, 9])
cm = sns.heatmap(df_cm, annot=True)
fig = cm.get_figure()

In [ ]:
# # We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
# mr = project.get_model_registry()

In [ ]:
# # The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
# model_dir="winequality_model"
# if os.path.isdir(model_dir) == False:
#     os.mkdir(model_dir)

In [ ]:
# # Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
# joblib.dump(model, model_dir + "/winequality_model.pkl")
# fig.savefig(model_dir + "/confusion_matrix.png")

In [ ]:
# # Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
# input_schema = Schema(X_train)
# output_schema = Schema(y_train)
# model_schema = ModelSchema(input_schema, output_schema)

In [ ]:
# # Create an entry in the model registry that includes the model's name, desc, metrics
# wine_model = mr.python.create_model(
#     name="winequality_model", 
#     metrics={"accuracy" : metrics['accuracy']},
#     model_schema=model_schema,
#     description="Wine Quality Predictor"
# )

In [ ]:
# # Upload the model to the model registry, including all files in 'model_dir'
# wine_model.save(model_dir)